**In this notebook I am using embedding of product descriptions to find product similarity**

This method might give some redundant results becuase many items have the same descriptions. But it might be useful in a hybrid recommendation system or for comparison of different models.

* Embeddings are produced by 'universal-sentence-encoder' found on TensorFlow Hub
* Distance metric used is dot product
* Similarity score are stored in a pickle file due to memory constraints 'https://www.kaggle.com/datasets/mohammedobeidat/product-desc-similarity-scores'



In [ ]:
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import pickle
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

path = '../input/h-and-m-personalized-fashion-recommendations/articles.csv'
 
df = pd.read_csv(path).astype(str)

In [ ]:
#@title Load the Universal Sentence Encoder's TF Hub module

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)

In [ ]:
desc = df['detail_desc'].unique()

In [ ]:
desc

In [ ]:
embeds = model(desc)

In [ ]:
with open('text_desc_embeddings.pickle', 'wb') as f:
    pickle.dump(embeds, f)
    
with open('text_desc.pickle', 'wb') as f:
    pickle.dump(desc, f)

In [ ]:
file = open('scores_top20.pkl','wb')

for embed in embeds: 
    top10 = np.inner(embed, embeds)
    top10_index = np.argsort(-top10)[:20]
    top10_score = top10[top10_index]

    pickle.dump([top10_index, top10_score], file)

file.close()

In [ ]:
def load_top10():
    
    top10_indecies = []
    top10_scores = []

    with open('../input/product-desc-similarity-scores/scores_top10.pkl', 'rb') as f:
        for i in range(len(desc)):
            try:
                row = pickle.load(f)
                top10_indecies.append(row[0])
                top10_scores.append(row[1])
            except:
                print('Done Loading')
                
    return top10_indecies, top10_scores

In [ ]:
top10_indecies, top10_scores = load_top10()

In [ ]:
def recommend(article_id):
    
    desc_list = desc.tolist()
    product_desc = df[df.article_id == sample]['detail_desc'].values[0]
    desc_index = desc_list.index(product_desc)
    rcmnds_indecies = top10_indecies[desc_index]
    rcmnds_scores = top10_scores[desc_index]
    rcmnds_descs = desc[rcmnds_indecies]
    map_dict = {i:j for i, j in zip(rcmnds_descs, rcmnds_scores)}
    rcmnds_article_ids = df[df.detail_desc.isin(rcmnds_descs)]
    rcmnds_article_ids['score'] = rcmnds_article_ids.detail_desc.map(map_dict)
    rcmnds_article_ids = rcmnds_article_ids[rcmnds_article_ids.score < 0.99]
    rcmnds_article_ids = rcmnds_article_ids.sort_values(by='score', ascending=False).drop_duplicates('score')
    
    
    return(rcmnds_article_ids[['article_id', 'score']])

In [ ]:
def plot_items(items):
    path = "../input/h-and-m-personalized-fashion-recommendations/images"

    k = len(items)
    fig = plt.figure(figsize=(15, 10))
    for item, i in zip(items, range(1, k+1)):
        item = '0' + str(item)
        sub = item[:3]
        image = path + "/"+ sub + "/"+ item +".jpg"
        image = plt.imread(image)
        fig.add_subplot(1, k, i)
        plt.imshow(image)
        

In [ ]:
sample = df.article_id.iloc[1]
rcmnds = recommend(sample)

In [ ]:
plot_items(rcmnds.sample(6).article_id.values)

In [ ]:
trans = next(pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv', dtype=str, chunksize=10000))

trans.drop_duplicates(['customer_id', 'article_id'], inplace=True)
trans.article_id = trans.article_id.map(lambda x: x[1:])
grouped = trans.groupby('customer_id')

In [ ]:
new = []
for group in grouped.groups:
    temp = grouped.get_group(group)
    if len(temp) >= 12:
        new.append([group, temp.article_id.values.tolist()[:12]])

In [ ]:
def get_sim(item, items):
    
    item_desc = df.detail_desc[df.article_id == item].values[0]
    item_embed = model([item_desc])[0]
    
    items_desc = df[df.article_id.isin(items)].detail_desc
    items_embed = model(items_desc)
    scores = []
    
    for i in items_embed:
        sim = np.dot(i, item_embed)
        scores.append(sim)
        
        
    return np.mean(scores)

In [ ]:
preds = []
scores = []

for key, value in new:
    temp = []
    for item in value[:6]:
        temp.append(recommend(item))
    temp2 = pd.concat(temp).sample(6, random_state=42)
    temp2['actual'] = value[6:]
    temp3 = []
    for item in temp2.actual:
        sim = get_sim(item, temp2.article_id) 
        temp3.append(sim)
        
    scores.append(np.mean(temp3))
    preds.append([key, temp2])

In [ ]:
np.mean(scores)

In [ ]:
customer = preds[0][1]

In [ ]:
customer

In [ ]:
plot_items(customer.article_id.values)

In [ ]:
plot_items(customer.actual.values)